<a href="https://colab.research.google.com/github/luccahonorio/pos-grad-spark/blob/main/AtividadeSpark_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade Spark

Considerando o dataset detalhado a seguir, extraia o conjunto de informações solicitadas.

### Dataset dados de COVID no Brasil

- Dados relativos a pacientes que realizaram exames de Covid19 no Brasil ate Marco 2020
- ~1.6GB
- 4.4M de instâncias


|  # | Nome do campo        	| Descrição                                     	|
|----|----------------------	|-----------------------------------------------	|
|  0 | id                   	| identificador                                 	|
|  1 | dataNotificacao      	| Data da notificação                           	|
|  2 | dataInicioSintomas   	| Data do inicio dos sintomas                   	|
|  3 | dataNascimento       	| Data de nascimento                            	|
|  4 | sintomas             	| Sintomas do paciente                          	|
|  5 | profissionalSaude    	| Relacionado a profissional de saúde           	|
|  6 | cbo                  	| Ocupação                                      	|
|  7 | condicoes            	| Condições do paciente                         	|
|  8 | estadoTeste          	| Estado do teste                               	|
|  9 | dataTeste            	| Data do teste                                 	|
| 10 | tipoTeste            	| Tipo de teste realizado                       	|
| 11 | resultadoTeste       	| Resultado do Teste                            	|
| 12 | paisOrigem           	| Pais de Origem do paciente                    	|
| 13 | sexo                 	| Sexo do paciente                              	|
| 14 | bairro               	| Bairro do paciente                            	|
| 15 | estado               	| Estado do paciente                            	|
| 16 | estadoIBGE              	| Estado do paciente IBGE                          	|
| 17 | municipio            	| Municipio do paciente                         	|
| 18 | municipioIBGE          	| Municipio do paciente                         	|
| 19 | cep            	        | CEP                                            	|
| 20 | origem               	| Origem do paciente                            	|
| 21 | cnes                 	| Código da unidade de saúde                    	|
| 22 | estadoNotificacao    	| Estado da notificação                         	|
| 23 | estadoNotificacaoIBGE    | Estado da notificação IBGE                      	|
| 24 | municipioNotificacao 	| Município da notificação                      	|
| 25 | municipioNotificacaoIBGE | Município da notificação IBGE                    	|
| 26 | numeroNotificacao    	| Número da notificação                         	|
| 27 | excluido             	| ID de exclusão                                	|
| 28 | validado             	| Local validação                               	|
| 29 | idade                	| Idade do paciente                             	|
| 30 | dataEncerramento     	| Data do encerramento da avaliação do paciente 	|
| 31 | evolucaoCaso         	| Evolução do caso do paciente                  	|
| 32 | classificacaoFinal   	| Avaliação final do caso                       	|

Informações a serem extraídas:

1. Quantidade de pacientes positivos para corona virus (resultadoTeste)
2. Quantidade de pacientes do sexo feminino positivos para corona virus (resultadoTeste)
3. Quantidade de pacientes de acordo com o sexo e o resultado do teste (resultadoTeste)
4. Sintomas mais comuns para casos positivos para corona virus
5. Sintomas mais comuns para casos não positivos para corona virus
6. Quantidade de casos positivos no Paraná
7. Municipio do Paraná com a maior quantidade de óbitos (evolucaoCaso)
8. Quantidade de municipios no Paraná com casos positivos
9. Quantidade de municipios no Paraná sem casos positivos
10. Estado com a maior taxa de falecimento de mulheres
11. Menor idade de mulher positiva para covid
12. Maior idade de mulher positiva para covid
13. Quantidade de casos positivos, em granularidade diária, ao longo do tempo
14. Quantidade de casos positivos, em granularidade semanal, ao longo do tempo
15. Quantidade de pacientes que faleceram (evolucaoCaso) separados pela idade
16. Taxa de óbito (evolucaoCaso) por idade
17. Idade média das mulheres positivas para covid
18. Município do Paraná com a maior quantidade de mulheres positivos para covid
19. Dia da semana com a maior quantidade de testes realizados
20. Dia da semana com maior quantidade de pacientes positivos para corona virus
21. Municipio, com mais de 500 testes, com a maior taxa de exames que não foram positivos para COVID (casos que foram testados porém não tiveram o resultadoTeste como Positivo)

**Dicas:**
- *Crie uma célula (Insert -> Insert Cell Below) para cada informação solicitada*

In [ ]:
a = 10
a = a + 2
print(a)

12


In [ ]:
#instala pyspark no google colab
!pip install pyspark==3.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 16.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845500 sha256=85bd2c7b68a5c5aad4855b7a0a878042b0c9fc495532e3243712a111a90475c0
  Stored in directory: /root/.cache/pip/wheels/0f/f0/3d/517368b8ce80486e84f89f214e0a022554e4ee64969f46279b
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
# abre sessão no spark
import pyspark
conf = pyspark.SparkConf()
conf.setMaster('local[*]')
sc = pyspark.SparkContext.getOrCreate(conf)

In [ ]:
#Efetua download do arquivo covid
!gdown 1EYMih3Ys1-DmaIZ9n0Sl5KAcRGqtpuj9

Downloading...
From: https://drive.google.com/uc?id=1EYMih3Ys1-DmaIZ9n0Sl5KAcRGqtpuj9
To: /content/covid.csv
100% 1.64G/1.64G [00:15<00:00, 106MB/s]


In [ ]:
#carrega arquivo da COVID em um rdd
rdd = sc.textFile('file:////content/covid.csv')

In [ ]:
def pegaSexo(linha):
  campos = linha.split(";")
  return campos[13]

def geraChaveValor(sexo):
  return [sexo, 1]

def somaValores(a,b):
  return a + b

rdd.map(pegaSexo).map(geraChaveValor).reduceByKey(somaValores).take(5)

[('Feminino', 2322760),
 ('null', 506),
 ('Indefinido', 107295),
 ('sexo', 1),
 ('Masculino', 1994043)]

In [ ]:
#informacao 1
def getPositivePacient(line):
  field = line.split(";")
  return [field[11],1]

def sumValues(a,b):
  return a + b

def removeFeel(entry):
  return entry[1] > 1000

def filterPositive(entry):
  return entry[0] == "Positivo"


rdd.map(getPositivePacient).reduceByKey(sumValues).filter(removeFeel).filter(filterPositive).collect()

[('Positivo', 1230210)]

In [ ]:
#informacao 2
def getPositivePacient(line):
  field = line.split(";")
  return [(field[11], field[13]), 1]

def sumValues(a,b):
  return a + b

def filterPositive(entry):
  return entry[0][0] == "Positivo" and entry[0][1] == "Feminino"

rdd.map(getPositivePacient).reduceByKey(sumValues).filter(filterPositive).collect()

# 632899

In [ ]:
#informacao 3


In [ ]:
#informacao 4


In [ ]:
#informacao 5


In [ ]:
#informacao 6


In [ ]:
#informacao 7


In [ ]:
#informacao 8


In [ ]:
#informacao 9


In [ ]:
#informacao 10


In [ ]:
#informacao 11


In [ ]:
#informacao 12


In [ ]:
#informacao 13


In [ ]:
#informacao 14


In [ ]:
#informacao 15


In [ ]:
#informacao 16


In [ ]:
#informacao 17


In [ ]:
#informacao 18


In [ ]:
#informacao 19


In [ ]:
#informacao 20


In [ ]:
#informacao 21
